# Minimal CANape Python `ctypes` Example

This notebook is a minimal example of using `ctypes` and `CANapeAPI` to call `Asap3GetVersion`.


## Example Goals:

- Load ```CANapAPI64.dll``` with ctypes.
- Read the library version.

## References:

- [```ctypes``` - A foreign function for Python](https://docs.python.org/3/library/ctypes.html)
- [CANape ASAM-MCD3 Interface Version Application Note AN-AMC-1-103](http://docplayer.net/56065428-Canape-asam-mcd3-interface-version-application-note-an-amc-1-103.html)
- ```CANapAPI.h``` (C:\Program Files\Vector CANape 19\CANapeAPI\CANapAPI.h)
        Description:
        |    ASAP3-Interface
        |    Version    : 1.0 28-AUG-2000
        |-----------------------------------------------------------------------------
        | Copyright (c) by Vector Informatik GmbH.  All rights reserved.

# Call Asap3GetVersion.

- Introduction to ```ctypes```
- Introduction to creating Python types from c types
- Creating Pythonic functions from c-functions.

The relevant portion from CANapAPI.h are copied below:

```
//! structure version_t used in function \link Asap3GetVersion \endlink
typedef struct {
  int dllMainVersion; //!< Main version of CANapeAPI DLL
  int dllSubVersion;  //!< Subversion version of CANapeAPI DLL
  int dllRelease;     //!< Release version of CANapeAPI DLL
  char osVersion[MAX_OS_VERSION];//!< Name of the minimum OS System
  int osRelease;      //!< minimum OS System version
} version_t;

extern bool Asap3GetVersion(version_t * version);
```

### Get Version. v0

- Minimal ctypes.
- Not Pythonic.
- Not fun to use.

In [1]:
import os
import ctypes
CANapAPI_dll = os.path.abspath(r"C:\Program Files\Vector CANape 19.0\CANapeAPI\CANapAPI64.dll")

In [2]:
# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)

# Define version_t as a Python ctypes-structure.
class version_t(ctypes.Structure):
    _fields_ = [
        ('dllMainVersion', ctypes.c_int),
        ('dllSubVersion', ctypes.c_int),
        ('dllRelease', ctypes.c_int),
        ('osVersion', ctypes.c_char * 50),
        ('osRelease', ctypes.c_int),
    ]

In [5]:
dll.Asap3GetVersion.restype = ctypes.c_bool

In [6]:
version=version_t()
result = dll.Asap3GetVersion(ctypes.byref(version))

In [7]:
result

True

In [8]:
version

*A man is flying in a hot air balloon and realizes he is lost. He spots a man down below and lowers the balloon to shout: “Excuse me, can you help me? I promised my friend I would meet him half an hour ago, but I don’t know where I am.”*

*The man below says: “Yes. You are in a hot air balloon, hovering approximately 30 feet above this field. You are between 40 and 42 degrees N. latitude, and between 58 and 60 degrees W. longitude.”*

*“You must be an engineer” says the balloonist.*

*“I am” replies the man. “How did you know.”*

*“Well” says the balloonist, “everything you have told me is technically correct, but I have no idea what to make of your information, and the fact is I am still lost.”*

# get_version. v1

- Make it Pythonic
- Advantages of High level language
  - High level languages are programmer friendly. ...
  - It provide higher level of abstraction from machine languages.
  - It is machine independent language.
  - Easy to learn.
  - Less error prone, easy to find and debug errors.
  - High level programming results in better programming productivity.

In [9]:
# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)
# Define version_t as a Python ctypes-structure.
class version_t(ctypes.Structure):
    _fields_ = [
        ('dllMainVersion', ctypes.c_int),
        ('dllSubVersion', ctypes.c_int),
        ('dllRelease', ctypes.c_int),
        ('osVersion', ctypes.c_char * 50),
        ('osRelease', ctypes.c_int),
    ]
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return str(other)==str(self)
        if isinstance(other, str):
            return str(other)==str(self)
        raise Exception(f"Unsupported class comparison {type(other)}")
    
    def __repr__(self):
        return f"API_VERSION<{self.dllMainVersion}.{self.dllSubVersion}.{self.dllRelease}>"

    def __str__(self):
        return "{}.{}.{}".format(self.dllMainVersion, self.dllSubVersion, self.dllRelease)

## Set the argument and return types.
# Pass by reference.
dll.Asap3GetVersion.argtypes = (ctypes.POINTER(version_t),)
# Return a success bool.
dll.Asap3GetVersion.restype = ctypes.c_bool

In [10]:
version = version_t()
result = dll.Asap3GetVersion(ctypes.byref(version))

In [11]:
result, version

(True, API_VERSION<2.3.1>)

In [12]:
version==version

True

In [13]:
version=="2.3.1"

True

In [14]:
print(version)

2.3.1


In [15]:
version.osVersion, version.osRelease

(b'Windows95/WindowsNT', 0)

In [16]:
version.dllMainVersion, version.dllSubVersion, version.dllRelease

(2, 3, 1)